In [125]:
# in this file there are functions to scrap the wikipedia webpages for cities 
# and push the info to sql database 
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy
from datetime import datetime
import re
from keys import MySQL_Pass

def create_sql_connection():
    schema = "sql_escooter_forecast"
    host = "127.0.0.1"
    user = "root"
    password = MySQL_Pass
    port = 3306
    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

    return connection_string

def get_city_info(urls):
    city_population_df = pd.DataFrame(columns=['city', 'population', 'date', 'lat' ,'long', 'country'])
    city_df = pd.DataFrame(columns=['city_id', 'city'])
    info_df = pd.DataFrame(columns=['city_id','population','date','lat','long'])
   # i=0
    for i, url in enumerate(urls):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        lat = soup.find('span', class_='latitude').get_text()
        longt = soup.find('span', class_='longitude').get_text()

        country_row0 = soup.find(lambda tag: tag.name == 'th' and 'Country' in tag.text)
        country = country_row0.find_next().get_text()

        population_row1 = soup.find(lambda tag: tag.name == 'th' and 'Population' in tag.text)
       # <div class="ib-settlement-fn"><span class="nowrap">&nbsp;</span>(2020)<sup id="cite_
        date = population_row1.find_next('div').get_text()
        population_row2 = population_row1.find_next('tr')
        population = int(population_row2.find_next('td').get_text().replace(',', ''))
        city = soup.find('div', class_='fn org').get_text()
        #i=i+1
        data_df = pd.DataFrame({'city_id':[int(i+1)],
                                'city': [city], 
                                'population': [population],
                                'date_retrieved':[date],
                                'lat': [lat],
                                'longt': [longt],
                                'country': [country]})
        city_population_df = pd.concat([city_population_df, data_df], ignore_index=True)
        info_df = city_population_df[['city_id','population','date_retrieved','lat','longt']].copy()
        city_df = city_population_df[['city_id','city']].copy()
        city_df['city_id'] = city_df['city_id'].astype(int)
        info_df['city_id'] = info_df['city_id'].astype(int)
        info_df['date_retrieved'] = info_df['date_retrieved'].apply(lambda x: datetime.
                                                strptime(re.search(r'\((\d{4}(?:-\d{2}-\d{2})?)\)', x).group(1), "%Y-%m-%d") if '-' in x else datetime.
                                                strptime(re.search(r'\((\d{4})\)', x).group(1), "%Y"))
        #city_df['city'][i] = city_df['city'][i].strip(" ")
        city_df['city'] = city_df['city'].str.strip()

    return city_df, info_df

def prepare_urls_for_cities(city_names):
    urls = ["https://en.wikipedia.org/wiki/" + city.replace(" ", "_") for city in city_names]
    return urls

def push_city_info_to_sql_table(city_df, connection_string, sql_table_name):
    city_df.to_sql(sql_table_name,
                   if_exists='append',
                   con=connection_string,
                   index=False)
    return 'Done Successfully!'

In [126]:
city_names = [
    "Berlin", 
    #"Hamburg", 
    #"Munich"
   #"Stuttgart",
    #"Cairo",
    #"Amman",
    #"São_Paulo",
    #"Mexico_City",
    #"Doha",
   # "Tunis",
   # "Algiers",
   # "Cologne",
   # "Frankfurt",
   # "Leipzig",
   # "Duesseldorf",
   #"Dortmund",
   # "Chicago",
   # "London",
  #  "Tripoli,_Libya"
    #"Paris"
    #"Alexandria"
]

In [127]:
urls = prepare_urls_for_cities(city_names)
city_df, info_df = get_city_info(urls)
connection_string = create_sql_connection()
push_city_info_to_sql_table(city_df, connection_string, 'cities')
push_city_info_to_sql_table(info_df, connection_string, 'facts')

'Done Successfully!'